In [1]:
import numpy as np
import random as rm

In [2]:
# The statespace
states = ["A", "T", "G", "C"]
# initial frequencies
initFreq = [0.25, 0.25, 0.25, 0.25]
# transition probabilities
transitionDic = {"A": [0.25, 0.25, 0.25, 0.25], 
                 "T": [0.4, 0.4, 0.1, 0.1], 
                 "G": [0.25, 0.25, 0.25, 0.25],
                 "C": [0.25, 0.25, 0.25, 0.25]}

n = 150
order = 1


seq = [np.random.choice(a= states, p = initFreq)]

for i in range(n - 1):
    n = ''.join(seq[-order:])
    prob = transitionDic[n]
    seq.append(np.random.choice(a= states, p = prob))


In [4]:
print(len(seq))

150


In [ ]:
# test probabilities sum to one
# test are all permutations in dict
# cases and controls
# encoding?